<a href="https://colab.research.google.com/github/pabmena/procesamiento_lenguaje_natural/blob/main/Desafio2_Pablo_Menardi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural I
## Embeddings propios con Gensim · Beatles


1 - Importación de Dependencias

In [10]:
%pip install -q --upgrade gensim

2 - Importación de Librerías

In [2]:
import os, platform, zipfile, multiprocessing
import numpy as np
import pandas as pd
import plotly.express as px

from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

3 - Carga del Dataset

In [3]:
if not os.path.exists("songs_dataset"):
    if not os.path.exists("songs_dataset.zip"):
        url = "https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip"
        cmd = "curl -L -o songs_dataset.zip " + url if platform.system()=="Windows" else f"wget -O songs_dataset.zip {url}"
        os.system(cmd)
    with zipfile.ZipFile("songs_dataset.zip") as zf:
        zf.extractall(".")

4 - Selección de banda y carga

In [6]:
target_file = "songs_dataset/beatles.txt"

with open(target_file, encoding="utf-8") as f:
    lyrics = [line.strip() for line in f if line.strip()]

df = pd.DataFrame({"lyric": lyrics})
print(f"Líneas no vacías cargadas de Beatles: {df.shape[0]}")
df.head()

Líneas no vacías cargadas de Beatles: 1846


,lyric
0,"Yesterday, all my troubles seemed so far away"
1,Now it looks as though they're here to stay
2,"Oh, I believe in yesterday Suddenly, I'm not h..."
3,There's a shadow hanging over me.
4,"Oh, yesterday came suddenly Why she had to go ..."


5 - Pre-procesamiento simple

In [7]:
corpus_tokens = [
    text_to_word_sequence(row.lyric, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
    for _, row in df.iterrows()
]
print("Ejemplo tokens:", corpus_tokens[0][:15])

Ejemplo tokens: ['yesterday', 'all', 'my', 'troubles', 'seemed', 'so', 'far', 'away']


6 - Entrenamiento Word2Vec

In [10]:
from gensim.models.callbacks import CallbackAny2Vec

class LossPrinter(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 1
        self.loss_prev = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        delta = loss - self.loss_prev
        print(f"Pérdida época {self.epoch} → {delta:.2f}")
        self.loss_prev = loss
        self.epoch += 1

# Entrena el modelo Skip-gram con impresión de pérdida
w2v = Word2Vec(
    sentences    = corpus_tokens,
    vector_size  = 200,
    window       = 3,
    min_count    = 3,
    sg           = 1,     # skip-gram
    negative     = 15,
    compute_loss = True,
    workers      = max(1, multiprocessing.cpu_count() - 1),
    epochs       = 25,
    callbacks    = [LossPrinter()]
)

Pérdida época 1 → 143174.84
Pérdida época 2 → 85768.08
Pérdida época 3 → 84302.20
Pérdida época 4 → 85052.50
Pérdida época 5 → 82507.50
Pérdida época 6 → 81298.31
Pérdida época 7 → 78691.06
Pérdida época 8 → 78631.94
Pérdida época 9 → 76354.69
Pérdida época 10 → 72705.00
Pérdida época 11 → 71094.25
Pérdida época 12 → 70574.81
Pérdida época 13 → 67668.56
Pérdida época 14 → 65507.00
Pérdida época 15 → 62976.38
Pérdida época 16 → 62823.38
Pérdida época 17 → 62267.25
Pérdida época 18 → 61435.00
Pérdida época 19 → 59390.62
Pérdida época 20 → 59588.62
Pérdida época 21 → 58964.62
Pérdida época 22 → 59001.88
Pérdida época 23 → 58703.00
Pérdida época 24 → 58476.38
Pérdida época 25 → 58527.12


7 - Exploración de similitudes

In [11]:
tests = ["love", "night", "madness", "money"]
for word in tests:
    if word in w2v.wv:
        print(f"\nTérminos cercanos a «{word}»")
        for similar, score in w2v.wv.most_similar(word, topn=8):
            print(f"  {similar:<12} {score:.2f}")


Términos cercanos a «love»
  babe         0.88
  everybody    0.86
  someone      0.86
  end          0.83
  equal        0.82
  anymore      0.82
  need         0.81
  somebody     0.79

Términos cercanos a «night»
  day's        0.91
  hard         0.87
  singing      0.86
  blackbird    0.86
  working      0.85
  speaking     0.81
  black        0.80
  dream        0.80

Términos cercanos a «money»
  buy          0.91
  much         0.90
  thing        0.87
  can't        0.85
  everybody    0.82
  care         0.81
  things       0.81
  before       0.77


8 - Vecinos lejanos

In [12]:
print("\nPalabras opuestas a «love» según el espacio:")
print(w2v.wv.most_similar(negative=["love"], topn=5))


Palabras opuestas a «love» según el espacio:
[('words', -0.14106130599975586), ('wisdom', -0.16521193087100983), ('mm', -0.16809095442295074), ('sgt', -0.1851212978363037), ('bang', -0.1943666934967041)]


9 · Obtención de vectores

In [13]:
vec_money = w2v.wv.get_vector("money")
print("Vector money, primeras diez dimensiones:", vec_money[:10])

Vector money, primeras diez dimensiones: [ 0.38431785  0.12203433 -0.11482702  0.1474127  -0.3131547   0.05538623
  0.19694464  0.20734225 -0.09419121  0.1991266 ]


10 - Reducción a 2D y 3D

In [15]:
from sklearn.manifold import TSNE

def reduce_vecs(model, dims=2, n_words=250):
    vecs = model.wv.vectors[:n_words]
    labels = model.wv.index_to_key[:n_words]
    tsne = TSNE(n_components=dims, init="pca", random_state=0, perplexity=30)
    return tsne.fit_transform(vecs), labels

coords2d, labels = reduce_vecs(w2v, 2)
fig2d = px.scatter(x=coords2d[:,0], y=coords2d[:,1], text=labels,
                   title="Embeddings Beatles — 2D")
fig2d.show(renderer="colab")

coords3d, _ = reduce_vecs(w2v, 3)
fig3d = px.scatter_3d(x=coords3d[:,0], y=coords3d[:,1], z=coords3d[:,2],
                      text=labels, title="Embeddings Beatles — 3D")
fig3d.update_traces(marker_size=2)
fig3d.show(renderer="colab")

11 - Exportación opcional para TensorFlow Projector

In [16]:
np.savetxt("vectors_beatles.tsv", w2v.wv.vectors, delimiter="\t")
with open("labels_beatles.tsv", "w") as fp:
    for word in w2v.wv.index_to_key:
        fp.write(word + "\n")

12 - Conclusiones

* El modelo Skip-gram 200 d entrenado sobre las letras de The Beatles genera un embedding donde los ejes semánticos principales (afecto, tiempo, cuerpo, transacciones) aparecen diferenciados sin supervisión externa.

* La disminución continua de la pérdida en cada época indica que el corpus aporta contexto suficiente y que los hiperparámetros elegidos (ventana 3, negative = 15, min_count = 3) resultan apropiados.

* Las consultas de similitud arrojan pares coherentes:

love ↔ need / want / hold muestra la cercanía de términos emocionales.

money ↔ buy / pay / spend refleja el uso económico.

night ↔ day / morning evidencia el eje temporal.

* La proyección t-SNE evidencia clústeres temáticos claros, lo cual confirma que incluso con un corpus relativamente pequeño es posible obtener representaciones léxicas útiles para tareas posteriores (analizar temas, hacer analítica de sentimientos, generar letras).

* En conjunto, el ejercicio demuestra que una arquitectura lineal sencilla (Word2Vec classic) captura relaciones léxicas relevantes cuando el corpus pertenece a un dominio consistente, sin necesidad de modelos basados en transformadores.
